# IBRD Statement of Loans ETL

# ⤵️ Import

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

# ⚒️ Function

In [2]:
def get_transform_data(
        input_path: str,
        source: str,
        out_path: str | None=None,
        dest: str | None=None
) -> pd.DataFrame:
    """
        Input the data from .csv file or from the api and
        transform it for a smooth import into either Excel, MySQL or MSSQLServer
        #### return pd.DataFrame
        
        Params
        ---
        - input_path: str
            Either the api or location of the .csv file.
        - source: str
            'api' if input_path is an api.
            'csv' if input_path is a .csv file.
        - out_path: str | None=None
            Location to save the .csv or .xlsx file, including name and extension.
            If None, data will be prepared for export through python-server connection.
        - dest: str | None=None
            If out_path is given, please choose dest.
            'ms' if destination is MSSQLServer.
            'my' if destination is MySQL.
            'xl' if destination is Excel.
            
        Example use
        ---
        
        `api = 'https://finances.worldbank.org/resource/sfv5-tf7p.csv?$limit=10000'`
        `out_path_mysql = 'ibrd_stmt_loans_data_transformed_mysql.csv'`
        `get_transform_data(api, source='api', csv_out_path=out_path_mysql, sql='my')`
    """

    # Importing

    df = pd.read_csv(input_path)
    print(f'Input  file: rows: {df.shape[0]}, columns: {df.shape[1]}')

    # Transforming

    # Column names to snake_case
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_',   regex=True)
    df.columns = df.columns.str.replace('(', '',    regex=True)
    df.columns = df.columns.str.replace(')', '',    regex=True)
    df.columns = df.columns.str.replace("'", '',    regex=True)
    df.columns = df.columns.str.replace('3', 'thi', regex=True)
    
    df.rename(columns={'effective_date_most_recent_': 'effective_date_most_recent',
                       'closed_date_most_recent_': 'closed_date_most_recent',
                       'borrower_s_obligation': 'borrowers_obligation'},
              inplace=True)
    
    # Fix issue with Namibia's country code 'NA' being placed as NaN by pd.read_csv()
    df['country_code'] = np.where(df['country'] == 'Namibia', 'NA', df['country_code'])
    df['guarantor_country_code'] = np.where(df['guarantor'] == 'Namibia', 'NA', df['guarantor_country_code'])

    # Format the date columns
    date_columns = ['end_of_period',
                    'first_repayment_date',
                    'last_repayment_date',
                    'agreement_signing_date',
                    'board_approval_date',
                    'effective_date_most_recent',
                    'closed_date_most_recent',
                    'last_disbursement_date']

    if source == 'csv':
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], format='%m/%d/%Y %H:%M:%S AM').dt.date
    elif source == 'api':
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%dT00:00:00.000').dt.date

    # due_third_party from int to float
    df['due_third_party'] = df['due_third_party'].astype(float)

    # interest rate from percentage to decimal
    df['interest_rate'] /= 100

    # Replace empty values (NaN) with 'NULL' (only for .csv files for MySQL)
    if dest == 'my' and out_path is not None:
        df.replace(np.nan, 'NULL', inplace=True)

    print(f'Output file: rows: {df.shape[0]}, columns: {df.shape[1]}')
    
    # Exporting

    if out_path is not None:
        # Export to .csv file
        if dest == 'ms' or dest == 'my':
            df.to_csv(out_path, index=False)
            if dest == 'ms':
                sql = 'MSSQLServer'
            else:
                sql = 'MySQL'
            print(f'Successfully exported transformed data as a csv file for {sql}')
            
        # Export to Excel
        if dest == 'xl':
            df.to_excel(out_path, index=False, sheet_name='data', freeze_panes=(1, 0))
            print('Successfully exported transformed data as an Excel file')
    else:
        print('Successfully prepared data for exporting through a connection with MySQL or MSSQLServer')

    return df

# 🪄 Parameters & Execution

In [3]:
csv = 'IBRD_Statement_of_Loans_-_Latest_Available_Snapshot.csv'
api = 'https://finances.worldbank.org/resource/sfv5-tf7p.csv?$limit=10000'

out_path_my = 'ibrd_stmt_loans_data_my.csv'
out_path_ms = 'ibrd_stmt_loans_data_ms.csv'
out_path_xl = 'ibrd_stmt_loans_data_xl.xlsx'

In [4]:
# get_transform_data(csv, source='csv', out_path=out_path_my, dest='my') # from csv to csv for mysql
# get_transform_data(csv, source='csv', out_path=out_path_ms, dest='ms') # from csv to csv for msssqlserver
get_transform_data(api, source='api', out_path=out_path_my, dest='my') # from api to csv for mysql
get_transform_data(api, source='api', out_path=out_path_ms, dest='ms') # from api to csv for msssqlserver
get_transform_data(api, source='api', out_path=out_path_xl, dest='xl') # from api to excel
data_export = get_transform_data(api, source='api') # from api to df for connection

Input  file: rows: 9196, columns: 33
Output file: rows: 9196, columns: 33
Successfully exported transformed data as a csv file for MySQL
Input  file: rows: 9196, columns: 33
Output file: rows: 9196, columns: 33
Successfully exported transformed data as a csv file for MSSQLServer
Input  file: rows: 9196, columns: 33
Output file: rows: 9196, columns: 33
Successfully exported transformed data as an Excel file
Input  file: rows: 9196, columns: 33
Output file: rows: 9196, columns: 33
Successfully prepared data for exporting through a connection with MySQL or MSSQLServer


# 🔗 Connect to MySQL

In [5]:
user = 'root'
password = 'root'
host = 'LocalHost'
database = 'ibrd_stmt_loans'

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [6]:
query_drop = """
DROP TABLE IF EXISTS import_data;    
"""

query_create = """
CREATE TABLE import_data (
    end_of_period 				DATE,
    loan_number 				VARCHAR(10),
    region 						VARCHAR(50),
    country_code 				VARCHAR(2),
    country 					VARCHAR(50),
    borrower 					VARCHAR(100),
    guarantor_country_code 		VARCHAR(2),
    guarantor 					VARCHAR(50),
    loan_type 					VARCHAR(5),
    loan_status 				VARCHAR(50),
    interest_rate 				DECIMAL(7, 6),
    currency_of_commitment 		VARCHAR(50),
    project_id 					VARCHAR(10),
    project_name 				VARCHAR(100),
    original_principal_amount 	DECIMAL(12, 2),
    cancelled_amount 			DECIMAL(12, 2),
    undisbursed_amount 			DECIMAL(12, 2),
    disbursed_amount 			DECIMAL(12, 2),
    repaid_to_ibrd 				DECIMAL(12, 2),
    due_to_ibrd 				DECIMAL(12, 2),
    exchange_adjustment 		DECIMAL(12, 2),
    borrowers_obligation 		DECIMAL(12, 2),
    sold_third_party 			DECIMAL(12, 2),
    repaid_third_party 			DECIMAL(12, 2),
    due_third_party 			DECIMAL(12, 2),
    loans_held 					DECIMAL(12, 2),
    first_repayment_date 		DATE,
    last_repayment_date 		DATE,
    agreement_signing_date 		DATE,
    board_approval_date 		DATE,
    effective_date_most_recent 	DATE,
    closed_date_most_recent 	DATE,
    last_disbursement_date 		DATE
);
"""

with engine.begin() as conn:
    conn.execute(query_drop)
    conn.execute(query_create)

In [7]:
# Export data to server
data_export.to_sql('import_data', con=engine, if_exists='append', index=False)

9196

In [8]:
# Check if the data is there
with engine.begin() as conn:
    df = pd.read_sql_query('SELECT * FROM import_data', conn)
    
df

,end_of_period,loan_number,region,country_code,country,borrower,guarantor_country_code,guarantor,loan_type,loan_status,...,repaid_third_party,due_third_party,loans_held,first_repayment_date,last_repayment_date,agreement_signing_date,board_approval_date,effective_date_most_recent,closed_date_most_recent,last_disbursement_date
0,2024-07-31,IBRD00010,EUROPE AND CENTRAL ASIA,FR,France,CREDIT NATIONAL,FR,France,NPL,Fully Repaid,...,2.499620e+08,0.0,0.0,1952-11-01,1977-05-01,1947-05-09,1947-05-09,1947-06-09,1947-12-31,None
1,2024-07-31,IBRD00020,EUROPE AND CENTRAL ASIA,NL,Netherlands,None,None,None,NPL,Fully Repaid,...,8.767200e+07,0.0,0.0,1952-04-01,1972-10-01,1947-08-07,1947-08-07,1947-09-11,1948-03-31,None
2,2024-07-31,IBRD00021,EUROPE AND CENTRAL ASIA,NL,Netherlands,None,None,None,NPL,Fully Repaid,...,3.955788e+06,0.0,0.0,1953-04-01,1954-04-01,1948-05-25,1947-08-07,1948-06-01,1948-06-30,None
3,2024-07-31,IBRD00030,EUROPE AND CENTRAL ASIA,DK,Denmark,None,None,None,NPL,Fully Repaid,...,2.222900e+07,0.0,0.0,1953-02-01,1972-08-01,1947-08-22,1947-08-22,1947-10-17,1949-03-31,None
4,2024-07-31,IBRD00040,EUROPE AND CENTRAL ASIA,LU,Luxembourg,None,None,None,NPL,Fully Repaid,...,1.014200e+07,0.0,0.0,1949-07-15,1972-07-15,1947-08-28,1947-08-28,1947-10-24,1949-03-31,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9191,2024-07-31,IBRDS0160,MIDDLE EAST AND NORTH AFRICA,DZ,Algeria,MINISTERE DES FINANCES,DZ,Algeria,NPL,Fully Cancelled,...,0.000000e+00,0.0,0.0,1984-05-15,1989-11-15,None,1979-11-13,1979-11-13,1983-06-30,None
9192,2024-07-31,IBRDS0170,MIDDLE EAST AND NORTH AFRICA,DZ,Algeria,MINISTERE DES FINANCES,DZ,Algeria,NPL,Fully Repaid,...,0.000000e+00,0.0,0.0,1983-11-15,1989-11-15,1980-02-14,1980-01-15,1980-09-17,1984-12-31,1985-11-13
9193,2024-07-31,IBRDS0180,MIDDLE EAST AND NORTH AFRICA,MA,Morocco,ONAREP,MA,Morocco,NPL,Fully Repaid,...,0.000000e+00,0.0,0.0,1984-11-01,1997-05-01,1980-05-19,1980-04-15,1980-10-24,1984-06-30,None
9194,2024-07-31,IBRDS0190,LATIN AMERICA AND CARIBBEAN,PE,Peru,EMPRESA SIDERURGICA DEL PERU,PE,Peru,NPL,Fully Repaid,...,0.000000e+00,0.0,0.0,1984-12-15,1990-12-15,1980-12-15,1980-07-15,1981-04-16,1986-06-30,1983-10-11


# 🔗 Connect to MSSQLServer

In [9]:
driver = 'SQL SERVER'
server = 'AASUS38A\SQLEXPRESS'
database = 'ibrd_stmt_loans'

connection_string = f"""
    DRIVER={driver};
    SERVER={server};
    DATABASE={database};
    Trusted_Connection=yes;
"""

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

In [10]:
query_drop = """
DROP TABLE IF EXISTS import_data;    
"""

query_create = """
CREATE TABLE import_data (
    end_of_period 				DATE,
    loan_number 				VARCHAR(10),
    region 						VARCHAR(50),
    country_code 				VARCHAR(2),
    country 					VARCHAR(50),
    borrower 					VARCHAR(100),
    guarantor_country_code 		VARCHAR(2),
    guarantor 					VARCHAR(50),
    loan_type 					VARCHAR(5),
    loan_status 				VARCHAR(50),
    interest_rate 				DECIMAL(7, 6),
    currency_of_commitment 		VARCHAR(50),
    project_id 					VARCHAR(10),
    project_name 				VARCHAR(100),
    original_principal_amount 	DECIMAL(12, 2),
    cancelled_amount 			DECIMAL(12, 2),
    undisbursed_amount 			DECIMAL(12, 2),
    disbursed_amount 			DECIMAL(12, 2),
    repaid_to_ibrd 				DECIMAL(12, 2),
    due_to_ibrd 				DECIMAL(12, 2),
    exchange_adjustment 		DECIMAL(12, 2),
    borrowers_obligation 		DECIMAL(12, 2),
    sold_third_party 			DECIMAL(12, 2),
    repaid_third_party 			DECIMAL(12, 2),
    due_third_party 			DECIMAL(12, 2),
    loans_held 					DECIMAL(12, 2),
    first_repayment_date 		DATE,
    last_repayment_date 		DATE,
    agreement_signing_date 		DATE,
    board_approval_date 		DATE,
    effective_date_most_recent 	DATE,
    closed_date_most_recent 	DATE,
    last_disbursement_date 		DATE
);
"""

with engine.begin() as conn:
    conn.execute(query_drop)
    conn.execute(query_create)

In [11]:
# Export data to server
data_export.to_sql('import_data', con=engine, if_exists='append', index=False)

-1

In [12]:
# Check if the data is there
with engine.begin() as conn:
    df = pd.read_sql_query('SELECT * FROM import_data', conn)
    
df

,end_of_period,loan_number,region,country_code,country,borrower,guarantor_country_code,guarantor,loan_type,loan_status,...,repaid_third_party,due_third_party,loans_held,first_repayment_date,last_repayment_date,agreement_signing_date,board_approval_date,effective_date_most_recent,closed_date_most_recent,last_disbursement_date
0,2024-07-31,IBRD00010,EUROPE AND CENTRAL ASIA,FR,France,CREDIT NATIONAL,FR,France,NPL,Fully Repaid,...,2.499620e+08,0.0,0.0,1952-11-01,1977-05-01,1947-05-09,1947-05-09,1947-06-09,1947-12-31,None
1,2024-07-31,IBRD00020,EUROPE AND CENTRAL ASIA,NL,Netherlands,None,None,None,NPL,Fully Repaid,...,8.767200e+07,0.0,0.0,1952-04-01,1972-10-01,1947-08-07,1947-08-07,1947-09-11,1948-03-31,None
2,2024-07-31,IBRD00021,EUROPE AND CENTRAL ASIA,NL,Netherlands,None,None,None,NPL,Fully Repaid,...,3.955788e+06,0.0,0.0,1953-04-01,1954-04-01,1948-05-25,1947-08-07,1948-06-01,1948-06-30,None
3,2024-07-31,IBRD00030,EUROPE AND CENTRAL ASIA,DK,Denmark,None,None,None,NPL,Fully Repaid,...,2.222900e+07,0.0,0.0,1953-02-01,1972-08-01,1947-08-22,1947-08-22,1947-10-17,1949-03-31,None
4,2024-07-31,IBRD00040,EUROPE AND CENTRAL ASIA,LU,Luxembourg,None,None,None,NPL,Fully Repaid,...,1.014200e+07,0.0,0.0,1949-07-15,1972-07-15,1947-08-28,1947-08-28,1947-10-24,1949-03-31,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9191,2024-07-31,IBRDS0160,MIDDLE EAST AND NORTH AFRICA,DZ,Algeria,MINISTERE DES FINANCES,DZ,Algeria,NPL,Fully Cancelled,...,0.000000e+00,0.0,0.0,1984-05-15,1989-11-15,None,1979-11-13,1979-11-13,1983-06-30,None
9192,2024-07-31,IBRDS0170,MIDDLE EAST AND NORTH AFRICA,DZ,Algeria,MINISTERE DES FINANCES,DZ,Algeria,NPL,Fully Repaid,...,0.000000e+00,0.0,0.0,1983-11-15,1989-11-15,1980-02-14,1980-01-15,1980-09-17,1984-12-31,1985-11-13
9193,2024-07-31,IBRDS0180,MIDDLE EAST AND NORTH AFRICA,MA,Morocco,ONAREP,MA,Morocco,NPL,Fully Repaid,...,0.000000e+00,0.0,0.0,1984-11-01,1997-05-01,1980-05-19,1980-04-15,1980-10-24,1984-06-30,None
9194,2024-07-31,IBRDS0190,LATIN AMERICA AND CARIBBEAN,PE,Peru,EMPRESA SIDERURGICA DEL PERU,PE,Peru,NPL,Fully Repaid,...,0.000000e+00,0.0,0.0,1984-12-15,1990-12-15,1980-12-15,1980-07-15,1981-04-16,1986-06-30,1983-10-11


---